# **Colab**

In [32]:
# === 1. Montagem do Google Drive e definição dos caminhos ===
from google.colab import drive
drive.mount('/content/drive')
caminho_base = '/content/drive/My Drive/Colab/cb-classificador/data/'
arquivo_entrada = caminho_base + 'item_nfe_rotulo.csv'
arquivo_tratado = caminho_base + 'dados_tratados.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# === 2. Instalação e importação de bibliotecas ===
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import csv
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_pt = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# === 3. Função para limpeza e padronização dos campos ===
def limpar_campos_textuais(df):
    # Tratamento de valores nulos
    df['cod_anp'] = df['cod_anp'].fillna('0')
    df['cod_ncm'] = df['cod_ncm'].fillna('0')
    df['des_anp'] = df['des_anp'].fillna('nihil')
    df['des_anp'] = df['des_anp'].replace(['0', 'ANP'], 'nihil')
    df['des_produto'] = df['des_produto'].fillna('nihil')
    df['cod_unidade'] = df['cod_unidade'].fillna('desconhecida')
    # Conversão para inteiros
    df['cod_anp'] = df['cod_anp'].apply(lambda x: int(float(x)) if x != '' else 0)
    df['cod_ncm'] = df['cod_ncm'].apply(lambda x: int(float(x)) if x != '' else 0)
    # Limpeza e padronização de descrições
    for col in ['des_anp', 'des_produto']:
        df[col] = df[col].astype(str)
        df[col] = df[col].str.replace(r'\\', '', regex=True)
        df[col] = df[col].str.replace(',', '-')
        df[col] = df[col].str.lower().apply(unidecode)
    # Padronização de unidades
    df['cod_unidade'] = df['cod_unidade'].astype(str).str.lower().apply(unidecode)
    return df

In [35]:
# === 4. Função para padronizar abreviaturas de códigos de unidade ===
def padronizar_codigos_unidade(df, coluna='cod_unidade'):
    mapa_cod_unidade = {
        'u': 'un',
        'un1': 'un',
        'und': 'un',
        'uni': 'un',
        'unid': 'un',
        'unidad': 'un',
        'l': 'lt',
        'l.': 'lt',
        'lts': 'lt',
        'lit': 'lt',
        'litro': 'lt',
        'h': 'hr',
        'hs': 'hr',
        'pç': 'pc',
        'peca': 'pc'
    }
    df[coluna] = df[coluna].replace(mapa_cod_unidade)
    return df

In [36]:
# === 5. Função para substituir valores com baixa frequência ===
def substituir_valores_raros(df, colunas_limites, valor_substituto=9):
    for coluna, limite in colunas_limites.items():
        frequencias = df[coluna].value_counts()
        raros = frequencias[frequencias < limite].index
        df[coluna] = df[coluna].apply(lambda x: valor_substituto if x in raros else x)
    return df

In [37]:
# === 6. Função para remoção de stopwords dos campos textuais ===
def remover_stopwords(texto, stopwords_lista):
    if pd.isnull(texto):  # Verificar se o valor não é nulo
        return texto
    palavras = texto.split()
    palavras_filtradas = [palavra for palavra in palavras if palavra not in stopwords_lista]
    return ' '.join(palavras_filtradas)

In [38]:
# === 7. Função para remoção linhas duplicadas ===
def remover_duplicatas(df):
    return df.drop_duplicates()

In [39]:
# === 8. Função para aplicar o pipeline de tratamento e padronização dos dados ===
def tratar_dados(df, colunas_e_limites, stopwords_lista):
    df = limpar_campos_textuais(df)
    df = padronizar_codigos_unidade(df)
    df = substituir_valores_raros(df, colunas_e_limites)
    df['des_anp'] = df['des_anp'].apply(lambda x: remover_stopwords(x, stopwords_lista))
    df['des_produto'] = df['des_produto'].apply(lambda x: remover_stopwords(x, stopwords_lista))
    return df

In [40]:
# === 9. Leitura do arquivo de entrada ===
df = pd.read_csv(arquivo_entrada, skiprows=1, header=None, quotechar='"', delimiter=',')
df.columns = ['categoria', 'ano', 'num_chave_nota_fiscal_eletronica', 'num_item_nota_fiscal_eletronica', 'cod_anp', 'cod_ncm', 'cod_unidade', 'des_anp', 'des_produto']
df = df[['categoria', 'cod_anp', 'cod_ncm', 'cod_unidade', 'des_anp', 'des_produto']]

In [41]:
len(df)

3695

In [42]:
df['categoria'].value_counts()

,count
categoria,
CB,2116
XX,1579


In [43]:
df['cod_ncm'].value_counts()

,count
cod_ncm,
27101259,1017
27101921,569
27101932,330
31021010,175
84212300,167
...,...
19049000,1
34070010,1
85124010,1


In [44]:
df['cod_anp'].value_counts()

,count
cod_anp,
320102001.0,503
0.0,326
820101034.0,258
320102002.0,233
820101033.0,119
620505001.0,118
810101001.0,97
320101001.0,27
820101012.0,20


In [45]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
UN,1385
L,1072
LT,746
PC,121
UNID,79
LITRO,76
l,49
UND,41
un,23


In [46]:
# === 10. Aplicar o pipeline de tratamento e padronização dos dados ===
colunas_e_limites = {
    'cod_ncm': 10,
    'cod_anp': 10,
    'cod_unidade': 5
}
df1 = tratar_dados(df, colunas_e_limites, stopwords_pt)

In [47]:
df1['cod_ncm'].value_counts()

,count
cod_ncm,
27101259,1017
27101921,569
9,345
27101932,330
31021010,175
84212300,167
22071090,120
22011000,115
84219999,110


In [48]:
df1['cod_anp'].value_counts()

,count
cod_anp,
0,2158
320102001,503
820101034,258
320102002,233
820101033,119
620505001,118
9,112
810101001,97
320101001,27


In [49]:
df1['cod_unidade'].value_counts()

,count
cod_unidade,
lt,1949
un,1564
pc,137
hr,17
9,15
dl,8
gl,5


In [50]:
# === 11. Remover duplicatas ===
df2 = remover_duplicatas(df1)

In [51]:
len(df2)

1914

In [52]:
df2['categoria'].value_counts()

,count
categoria,
XX,1110
CB,804


In [53]:
df2['cod_ncm'].value_counts()

,count
cod_ncm,
9,290
27101932,259
27101921,236
27101259,220
84212300,118
84219999,87
22011000,80
31021010,74
34029039,41


In [54]:
df2['cod_anp'].value_counts()

,count
cod_anp,
0,1363
820101034,106
620505001,88
320102001,76
9,74
320102002,74
820101033,43
810101001,32
740101006,15


In [55]:
df2['cod_unidade'].value_counts()

,count
cod_unidade,
un,1197
lt,615
pc,73
9,13
hr,10
gl,5
dl,1


In [56]:
# === 14. Armazenar arquivo tratado ===
df2.to_csv(arquivo_tratado, index=False, quoting=csv.QUOTE_NONE, escapechar='\\')